In [174]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import random
from compute_score import pearson_score, euclidean_score

Imported libraries:
- pandas to read and manage data files
- sklearn.neighbors.NearestNeighbors - K Nearest Neighbors model to predict other movies that are similar
- random - pick a random value

In [175]:
df_movie = pd.read_csv('movie.csv')
df_rating = pd.read_csv('rating.csv')
df_tag = pd.read_csv('tag.csv')

Use pandas to read movie.csv, rating.csv, and tag.csv and store them in df_movie, df_rating, and df_tag.

In [176]:
unique_movie = df_movie['movieId'].unique()

Get a list of unique movie ids and store them in unique_movie.
This is to make sure there are no duplicate movies in the csv files

In [177]:
unique_tag_list = []
for t in df_movie['genres']:
    temp = t.split('|')
    for word in temp:
        if word not in unique_tag_list:
            unique_tag_list.append(word)

unique_tag_list.remove('IMAX')

Go through all genres of all the movies and store them in a list. This is the list of all tags that are in movie.csv 
- ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'Mystery', 'Sci-Fi', 'IMAX', 'Documentary', 'War', 'Musical', 'Western', 'Film-Noir', '(no genres listed)']

I noticed that in 'IMAX' is in the list so I decided to remove it because it is not a genre. This is now the current genres in the list:
- ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'Mystery', 'Sci-Fi', 'Documentary', 'War', 'Musical', 'Western', 'Film-Noir', '(no genres listed)']

In [178]:
genre = {}
for t in unique_tag_list:
    genre[t] = 0

In [ ]:
print(genre)

I use dictionary to store all genres in unique_tag_list as key and with the values as 0. This is what the dictionary genre look like:
- {'Adventure': 0, 'Animation': 0, 'Children': 0, 'Comedy': 0, 'Fantasy': 0, 'Romance': 0, 'Drama': 0, 'Action': 0, 'Crime': 0, 'Thriller': 0, 'Horror': 0, 'Mystery': 0, 'Sci-Fi': 0, 'Documentary': 0, 'War': 0, 'Musical': 0, 'Western': 0, 'Film-Noir': 0, '(no genres listed)': 0}

In [179]:
X = {}
for m in unique_movie:
    X[str(m)] = genre.copy()

I make a dictionary called X. For all movie id in unique_movie as key, I assigned the value as genre. So in this list X contain the key value of all movie id, and with it's value is a genre dictionary with all values in it as 0.

In [180]:
for movie in X:
    temp = str((df_movie.loc[df_movie['movieId'] == int(movie)])['genres']).split()[1]
    temp = temp.split('|')
    for g in temp:
        if g in X[movie]:
            X[movie][g] = 1
    X[movie]['movieId'] = movie

For all key value in X is a movieId, and the value of each movieId are genre dictionary.<br/>
I search df_movie for genre, and for each genre in df_movie for all movies I flip genre dictionary value to one.
I also store each movieId in each X dictionary value.

<br/>
ex. {'Adventure': 0, 'Animation': 0, 'Children': 0, 'Comedy': 0, 'Fantasy': 0, 'Romance': 0, 'Drama': 0, 'Action': 0, 'Crime': 0, 'Thriller': 0, 'Horror': 0, 'Mystery': 0, 'Sci-Fi': 0, 'Documentary': 0, 'War': 0, 'Musical': 0, 'Western': 0, 'Film-Noir': 0, '(no genres listed)': 0, 'movieId': '1'} <br/> ==> <br/>{'Adventure': 1, 'Animation': 1, 'Children': 1, 'Comedy': 1, 'Fantasy': 1, 'Romance': 0, 'Drama': 0, 'Action': 0, 'Crime': 0, 'Thriller': 0, 'Horror': 0, 'Mystery': 0, 'Sci-Fi': 0, 'Documentary': 0, 'War': 0, 'Musical': 0, 'Western': 0, 'Film-Noir': 0, '(no genres listed)': 0, 'movieId': '1'}

In [181]:
X_list = []
X_list_id = []
for i, x in enumerate(X):
    X_list.append(list(X[x].values())[0:-1])
    X_list_id.append(list(X[x].values())[-1])

I converted all values in each movieId in X dictionary to list. I append the list except for the last element in the list in X_list, append the last element in the list into X_list_id. X_list contains lists of all genres value that converted from {'Adventure': 1, 'Animation': 1, 'Children': 1, 'Comedy': 1, 'Fantasy': 1, 'Romance': 0, 'Drama': 0, 'Action': 0, 'Crime': 0, 'Thriller': 0, 'Horror': 0, 'Mystery': 0, 'Sci-Fi': 0, 'Documentary': 0, 'War': 0, 'Musical': 0, 'Western': 0, 'Film-Noir': 0, '(no genres listed)': 0, 'movieId': '1'} to [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
<br/>
X_list_id is a list of all movie Id. X_list and X_list_id have the same number of all elements. I will use X_list as input for knn model.
- X_list = [[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], ...]
- X_list_id = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15',....]

In [182]:
random_movie = random.choice(list(X.values()))
y_movie_id = list(random_movie.values())[-1]
y_movie_name = df_movie.loc[df_movie['movieId'] == int(y_movie_id)]['title'].to_string(index=False)
y_movie_genre = df_movie.loc[df_movie['movieId'] == int(y_movie_id)]['genres'].to_string(index=False)
y_movie = list(random_movie.values())[0:-1]

print('Picked random movie named:', y_movie_name, 'genres', y_movie_genre)


Picked random movie named: Cellar, The (Huset vid vägens ände) (2003) genres Horror|Thriller


I use random to pick a random movieId in X. That random movie is my Value.

In [183]:
k = 5
knn = NearestNeighbors(n_neighbors=k).fit(X_list)
distances, indices = knn.kneighbors([y_movie])

print('Other movies similar to the movie', y_movie_name, 'genres', y_movie_genre, 'are')
for rank, index in enumerate(indices[0][:k], start=1):
    m_name = df_movie.loc[df_movie['movieId'] == int(X_list_id[index])]['title'].to_string(index=False)
    m_genre = df_movie.loc[df_movie['movieId'] == int(X_list_id[index])]['genres'].to_string(index=False)
    print('Rank', str(rank) + " ==> movie name:", m_name, m_genre)
    


Other movies similar to the movie Cellar, The (Huset vid vägens ände) (2003) genres Horror|Thriller are
Rank 1 ==> movie name: And Now the Screaming Starts! (1973) Horror|Thriller
Rank 2 ==> movie name: House at the End of the Street (2012) Horror|Thriller
Rank 3 ==> movie name: Session 9 (2001) Horror|Thriller
Rank 4 ==> movie name: Quicksilver Highway (1997) Horror|Thriller
Rank 5 ==> movie name: Madhouse (2004) Horror|Thriller


I use NearestNeighbors as a model, and fit X_list. Then I use kneighbors() to get 5 movies that are similar to the movie that I randomly picked. I left all parameters as default except for number of neighbors. I'm using KNN to reccommend 5 other movies that are similar to a movie that was randomly picked based on their genres.
<br/>
result:<br/>
Other movies similar to the movie Cellar, The (Huset vid vägens ände) (2003) genres Horror|Thriller are<br/>
Rank 1 ==> movie name: And Now the Screaming Starts! (1973) Horror|Thriller<br/>
Rank 2 ==> movie name: House at the End of the Street (2012) Horror|Thriller<br/>
Rank 3 ==> movie name: Session 9 (2001) Horror|Thriller<br/>
Rank 4 ==> movie name: Quicksilver Highway (1997) Horror|Thriller<br/>
Rank 5 ==> movie name: Madhouse (2004) Horror|Thriller<br/>

In [184]:
X2 = {}
for m in unique_movie:
    X2[str(m)] = genre.copy()

for movie in X2:
    temp = str((df_movie.loc[df_movie['movieId'] == int(movie)])['genres']).split()[1]
    temp = temp.split('|')
    for g in temp:
        if g in X2[movie]:
            X2[movie][g] = 1


I want to confirm that the recomendations by using pearson and euclidean scores for the performance metric. So this steps I altered the X data a bit to be used in pearson and euclidean score method.

In [186]:
print('Comparing movie ', y_movie_name)
for rank, index in enumerate(indices[0][:k], start=1):
    user1 = random_movie['movieId']
    user2 = X_list_id[index]
    m_name = df_movie.loc[df_movie['movieId'] == int(X_list_id[index])]['title'].to_string(index=False)
    print('Movie name:', m_name, 'Pearson Score = ', pearson_score(X2, user1, user2), 'Euclidean Score = ', euclidean_score(X2, user1, user2))

Comparing movie  Cellar, The (Huset vid vägens ände) (2003)
Movie name: And Now the Screaming Starts! (1973) Pearson Score =  1.0 Euclidean Score =  1.0
Movie name: House at the End of the Street (2012) Pearson Score =  1.0 Euclidean Score =  1.0
Movie name: Session 9 (2001) Pearson Score =  1.0 Euclidean Score =  1.0
Movie name: Quicksilver Highway (1997) Pearson Score =  1.0 Euclidean Score =  1.0
Movie name: Madhouse (2004) Pearson Score =  1.0 Euclidean Score =  1.0


I compared the random movie picked to the recomended movie using pearson and euclidean score. The scores that are got are all 1.0 for all the movies that I have recomended based on genre.
<br/>
Result:<br/>
Comparing movie  Cellar, The (Huset vid vägens ände) (2003)<br/>
Movie name: And Now the Screaming Starts! (1973) Pearson Score =  1.0 Euclidean Score =  1.0<br/>
Movie name: House at the End of the Street (2012) Pearson Score =  1.0 Euclidean Score =  1.0<br/>
Movie name: Session 9 (2001) Pearson Score =  1.0 Euclidean Score =  1.0<br/>
Movie name: Quicksilver Highway (1997) Pearson Score =  1.0 Euclidean Score =  1.0<br/>
Movie name: Madhouse (2004) Pearson Score =  1.0 Euclidean Score =  1.0<br/>